In [9]:
# 기본
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 경고 뜨지 않게 설정
import warnings
warnings.filterwarnings('ignore')

# 그래프 설정
sns.set()

# 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False

# 결측치 시각화를 위한 라이브러리
!pip install missingno
import missingno

# 데이터 전처리 알고리즘
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# 학습용과 검증용으로 나누는 함수
from sklearn.model_selection import train_test_split

# 교차 검증
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

# 평가함수
# 분류용
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

# 회귀용
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# 모델의 최적의 하이퍼 파라미터를 찾기 위한 도구
from sklearn.model_selection import GridSearchCV

# 머신러닝 알고리즘 - 분류
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier

# 머신러닝 알고리즘 - 회귀
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import VotingRegressor

# 학습 모델 저장을 위한 라이브러리
import pickle

### 1. train 데이터와 test 데이터로 분리한다.

In [12]:
r05 = pd.read_parquet('/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/인공지능 모델 구축/r05.parquet')
r05.head()


,이용금액_R3M_신용체크,이용금액_R3M_신용,_1순위카드이용금액,기준년월,ID,Segment,정상청구원금_B0M,정상청구원금_B2M,이용금액_일시불_R12M,이용금액_일시불_B0M,...,이용금액_오프라인_B0M,정상입금원금_B5M,정상입금원금_B0M,이용금액_오프라인_R3M,이용금액_오프라인_R6M,정상입금원금_B2M,_3순위업종_이용금액,청구금액_R6M,청구금액_R3M,청구금액_B0
0,196,196,3681,201807,TRAIN_000000,D,14440,16524,20667,1995,...,4043,9205,6335,4131,11097,16125,672,88693,46588,12226
1,196,196,3681,201807,TRAIN_000000,D,14440,16524,20667,1995,...,4043,9205,6335,4131,11097,16125,672,119419,53336,15756
2,196,196,3681,201807,TRAIN_000000,D,14440,16524,20667,1995,...,4043,9205,6335,4131,11097,16125,672,132269,43715,15732
3,196,196,3681,201807,TRAIN_000000,D,14440,16524,20667,1995,...,4043,9205,6335,4131,11097,16125,672,92304,45716,14227
4,196,196,3681,201807,TRAIN_000000,D,14440,16524,20667,1995,...,4043,9205,6335,4131,11097,16125,672,97254,43918,13958


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [13]:
# 2. 특성(X)과 타겟(y) 분리
TARGET_COLUMN_NAME = 'Segment'

# 타겟 컬럼이 데이터프레임에 존재하는지 확인 (오류 방지)
if TARGET_COLUMN_NAME not in r05.columns:
    raise ValueError(f"'{TARGET_COLUMN_NAME}' 컬럼이 r05 데이터프레임에 존재하지 않습니다. 실제 타겟 컬럼 이름을 정확히 입력해주세요.")

X = r05.drop(columns=[TARGET_COLUMN_NAME])  # 타겟 컬럼을 제외한 모든 컬럼이 특성 데이터
y = r05[TARGET_COLUMN_NAME]                  # 타겟 컬럼만 타겟 데이터

print(f"\n--- 특성(X) 및 타겟(y) 분리 완료 ---")
print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")

# 중요: 타겟 컬럼(y)이 문자열 범주형인 경우 Label Encoding이 필요합니다.
# 이미 숫자로 인코딩되어 있다면 이 부분은 건너뛰세요.
if y.dtype == 'object' or pd.api.types.is_categorical_dtype(y):
    le = LabelEncoder()
    y_encoded = le.fit_transform(y)
    print(f"\n--- 타겟 변수 Label Encoding 완료 ({le.classes_}) ---")
    print(f"원본 y head: {y.head()}")
    print(f"인코딩된 y head: {pd.Series(y_encoded).head()}")
else:
    y_encoded = y # 이미 숫자형인 경우 그대로 사용
    print(f"\n--- 타겟 변수는 이미 숫자형입니다. Label Encoding 건너_었습니다. ---")


# 3. 훈련(Train) 세트와 테스트(Test) 세트로 분할
# test_size=0.2: 전체 데이터의 20%를 테스트 세트로 사용 (80%는 훈련 세트)
# random_state=42: 매번 동일한 결과를 얻기 위한 난수 시드 고정 (재현성 확보)
# stratify=y_encoded: y (타겟 변수)의 클래스 비율을 훈련 세트와 테스트 세트 모두에 동일하게 유지
#                     (특히 클래스 불균형 데이터에 매우 중요합니다)
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)


print("\n--- 데이터 분할 결과 ---")
print(f"훈련 데이터 (X_train) shape: {X_train.shape}")
print(f"테스트 데이터 (X_test) shape: {X_test.shape}")
print(f"훈련 타겟 (y_train) shape: {y_train.shape}")
print(f"테스트 타겟 (y_test) shape: {y_test.shape}")


# 4. 분할된 데이터를 각각 Parquet 파일로 저장
# 훈련 데이터프레임 재구성 (X_train과 y_train 합치기)
# 합칠 때는 y_train (인코딩된 값)을 사용해야 합니다.
df_train = pd.concat([X_train, pd.Series(y_train, index=X_train.index, name=TARGET_COLUMN_NAME)], axis=1)
# 테스트 데이터프레임 재구성 (X_test와 y_test 합치기)
df_test = pd.concat([X_test, pd.Series(y_test, index=X_test.index, name=TARGET_COLUMN_NAME)], axis=1)


df_train.to_parquet('/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/인공지능 모델 구축/r05_train.parquet', index=False)
df_test.to_parquet('/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/인공지능 모델 구축/r05_test.parquet', index=False)


print("\n데이터가 '/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/인공지능 모델 구축/r05_train.parquet'와 'r05_test.parquet' 파일로 성공적으로 저장되었습니다.")


--- 특성(X) 및 타겟(y) 분리 완료 ---
X shape: (86400000, 21)
y shape: (86400000,)

--- 타겟 변수 Label Encoding 완료 (['A' 'B' 'C' 'D' 'E']) ---
원본 y head: 0    D
1    D
2    D
3    D
4    D
Name: Segment, dtype: category
Categories (5, object): ['A', 'B', 'C', 'D', 'E']
인코딩된 y head: 0    3
1    3
2    3
3    3
4    3
dtype: int64

--- 데이터 분할 결과 ---
훈련 데이터 (X_train) shape: (69120000, 21)
테스트 데이터 (X_test) shape: (17280000, 21)
훈련 타겟 (y_train) shape: (69120000,)
테스트 타겟 (y_test) shape: (17280000,)

데이터가 '/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/인공지능 모델 구축/r05_train.parquet'와 'r05_test.parquet' 파일로 성공적으로 저장되었습니다.
